# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Откройте файл с данными и изучите общую информацию

Импортируем библиотеку `pandas` и модуль `Mystem`, с которыми мы будем работать:

In [1]:
import pandas as pd
import numpy as np
from pymystem3 import Mystem

### Чтение файла
Прочитаем файл и выведем пять первых записей:

In [2]:
df = pd.read_csv('datasets/data.csv')
df.head()
def observe_data(table):
    '''Giving common understanding of data in dataframe'''
    print(f'Количество записей в таблице: {table.shape[0]} \nKоличество столбцов в таблице: {table.shape[1]}')
    print('_'*100)
    names_columns = table.columns
    print(f'Названия столбцов: {names_columns}')
    print('_'*100)
    names_to_corrige = []
    for name_column in names_columns:
        if not name_column.islower() or ' ' in name_column:
            names_to_corrige.append(name_column)
    if len(names_to_corrige) == 0:
        print (f'Названия столбцов корректны')
    else:
        print(f'Названия следующих колонок должны быть откорретированы: {names_to_corrige}')
    print('_'*100)
    lost_values = table.isna().sum()
    missed_values = lost_values.to_frame(name='missed_values')
    missed_values['percent'] = round(missed_values['missed_values']/table.shape[0]*100, 0)
    print (f'Пропущенные значения в абсолютном и относительном выражениях:')
    display(missed_values)
    print('_'*100)
    print(f'Типы данных в колонках:')
    print(table.dtypes)

     
        
observe_data(df)


Количество записей в таблице: 21525 
Kоличество столбцов в таблице: 12
____________________________________________________________________________________________________
Названия столбцов: Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')
____________________________________________________________________________________________________
Названия столбцов корректны
____________________________________________________________________________________________________
Пропущенные значения в абсолютном и относительном выражениях:


,missed_values,percent
children,0,0.0
days_employed,2174,10.0
dob_years,0,0.0
education,0,0.0
education_id,0,0.0
family_status,0,0.0
family_status_id,0,0.0
gender,0,0.0
income_type,0,0.0
debt,0,0.0


____________________________________________________________________________________________________
Типы данных в колонках:
children              int64
days_employed       float64
dob_years             int64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income        float64
purpose              object
dtype: object


Ради интереса посмотрим клиента с наибольшим месячным доходом и цель кредитования:

In [3]:
print(df['total_income'].max())
df[df['total_income'] == df['total_income'].max()]['purpose']


2265604.028722744


12412    ремонт жилью
Name: purpose, dtype: object

In [4]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**Вывод** 
- таблица содержит 21525 записей
- названия столбцов написаны корректно, нет необходимости менять их имена
- типы столбцов соответствуют их значениям: столбцы с количественными переменными (`children`, `days employed`, `dob_years`, `debt`,`total_income`) имеют тип `float` или `int`, в то время как оставшиеся столбцы с категориальными переменными имеют тип `object` за исключением столбцов `education_id` и `family_status_id`, имеющих тип `int`. Исходя из первичных представлений нет необходимости менять тип данных в столбцах помимо столбцов с типом данных `float` - для них такая точность значений не нужна, и для более читабельного представления данных в дальнейшем можно поменять их тип на `int`.
- пропуски данных для столбцов days_employed и total_income в количестве 21525-19351 = 2174, что составляет 10% от общего числа записей
- кто этот человек с ежемесячным доходом больше 2х миллионов и зачем ему вообще кредит, и вообще почему 'ремонт жилью', а не 'ремонт жилья'?

### Корректность данных в столбцах

Чтобы убедиться в корректности предоставленной информации (например, значения в столбцах `children`, `days employed` могут быть только положительными или равными 0; значения в столбце `dob_years` скорее всего должны быть больше 18 (лет), `gender` - только F или M и т.д.). Для этого применим метод `value_counts()` или `describe()`:

#### Колонка `children`

In [5]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Вывод Как видим в 47 записях фигурирует отрицательное количество детей (-1), что невозможно (если только банк не собирает информацию об умерших детях), также стоит обратить внимание на 76 клиентов с 20 детьми в семье. Скорее всего это техническая ошибка при выгрузке данных. Суммарно количество артефактов по столбцу children составляет 47+76=123 записи или 0,6% от общего числа записей и не окажет существенного влияния на определяемые показатели. Тем не менее, откорректируем часть данных, заменив -1 на 1 и 20 на 2:

In [6]:
df['children'] = df['children'].apply(lambda x: abs(x))
df.loc[df['children'] == 20, 'children'] = 2

Проверим изменения:

In [7]:
df['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

#### Колонка `days_employed`

In [8]:
display(df['days_employed'].describe())
negative_days_employed = df[df['days_employed'] < 0]
total_negative_days_employed = negative_days_employed.shape[0]

print(f'Количество всех записей в таблице с отрицательным значением трудового стажа составляет: {total_negative_days_employed} ')

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

Количество всех записей в таблице с отрицательным значением трудового стажа составляет: 15906 


**Вывод** Как и в колонке `children` общий трудовой стаж не может быть отрицательным числом. Из-за большого количества таких записей (более 70% от всей информации) можно предположить что при выгрузке данных имела место техническая ошибка. Для дальнейшего анализа информации заменим отрицательные значения в `days_employed` на положительные и проверим как изменились данные:

In [9]:
df['days_employed'] = df['days_employed'].apply(lambda x: abs(x))
df['days_employed'].describe()

count     19351.000000
mean      66914.728907
std      139030.880527
min          24.141633
25%         927.009265
50%        2194.220567
75%        5537.882441
max      401755.400475
Name: days_employed, dtype: float64

#### Колонка `dob_years`

Логично предположить, что клиент должен быть старше 18 лет, в то время как минимальный возраст клиента, указанный в таблице равен 0. Найдем количество таких записей, где возраст клиента не соответствует нашим ожиданиям

In [10]:
row_age_less_18 = df[df['dob_years'] < 18].shape[0]
row_age_zero = df[(df['dob_years'] == 0)].shape[0]
print(f'Количества клиентов, недостигших совершеннолетия: {row_age_less_18}')
print(f'Количества клиентов, возраст которых равен 0: {row_age_zero}')

Количества клиентов, недостигших совершеннолетия: 101
Количества клиентов, возраст которых равен 0: 101


Как мы видим, 101 запись содержит возраст клиента равный 0. Скорее всего, в данном случае 0 использовался для заполнения недостающих данных о возрасте. Для корректировки данных, заменим 0 в данных записях на значение медианы возраста клиентов, используя метод `.loc()`  и проверим значения:

In [11]:
for inc_type in df['income_type'].unique():
    median = df.loc[df['income_type'] == inc_type, 'dob_years'].median()
    df.loc[(df['income_type'] == inc_type) & (df['dob_years'] == 0), 'dob_years'] = median
df['dob_years'].describe()   

count    21525.000000
mean        43.496167
std         12.231538
min         19.000000
25%         34.000000
50%         43.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

**Вывод** 101 запись содержит возраст клиента равный 0. Скорее всего, в данном случае 0 использовался для заполнения недостающих данных о возрасте. Для корректировки данных, заменили 0 в данных записях на значение медианы возраста клиентов.

#### Колонка `education`

In [12]:
df['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

**Вывод** Значения данного столбца будут откорректированы в пунке 2.3 

#### Колонка `education_id`

По данным колонки `education` видно, что у нас может быть 5 категорий образования (начальное, среднее, незаконченное высшее, высшее, ученая степень):

In [13]:
df['education_id'].value_counts()

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

**Вывод** Данные корректны, полученный результат соответсвует ожидаемому

#### Колонка `family_status`

In [14]:
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

**Вывод** Данные корректны и дают 5 различных категорий

#### Колонка `family_status_id`

In [15]:
df['family_status_id'].value_counts()

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

**Вывод** Данные корректны и представлены 5 категориями

#### Колонка `gender`

In [16]:
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Очень интересно, применяют ли банку систему самоидентификации по половому признаку, поскольку одна запись имеет пол XNA, что бы это не значило. Поскольку мы не можем заполнить эту категориальную переменную по имеющимся у нас данным заменим XNA на `unknown`:

In [17]:
df.loc[df['gender'] == 'XNA', 'gender'] = 'unknown'
df['gender'].value_counts()

F          14236
M           7288
unknown        1
Name: gender, dtype: int64

**Вывод** Переменная ¨XNA заменена на "unknown"

#### Колонка `income_type`

In [18]:
df['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

**Вывод** Данные корректны

#### Колонка `debt`

In [19]:
df['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

**Вывод** Данные корректны, 0 - нет задолженности, 1 - имеется задолженность

#### Колонка `total_income`

In [20]:
df['total_income'].describe()

count    1.935100e+04
mean     1.674223e+05
std      1.029716e+05
min      2.066726e+04
25%      1.030532e+05
50%      1.450179e+05
75%      2.034351e+05
max      2.265604e+06
Name: total_income, dtype: float64

**Вывод** Имеющиеся данные корректны, артефактов не обнаружено.

#### Колонка `purpose`


In [21]:
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Вывод** Одни и те же цели формулируются клиентами по разному, в дальнейшем мы проведем лемматизацию значений этого столбца

**Вывод**
- таблица содержит 21525 записей
- названия столбцов написаны корректно, нет необходимости менять их имена
- типы столбцов соответствуют их значениям: столбцы с количественными переменными (`children`, `days employed`, `dob_years`, `debt`,`total_income`) имеют тип `float` или `int`в то время как оставшиеся столбцы с категориальными переменными имеют тип `object` за исключением столбцов `education_id ` и `family_status_id`, имеющих тип `int`. Исходя из первичных представлений нет необходимости менять тип данных в столбцах помимо столбцов с типом данных `float` - для них такая точность значений не нужна, и для более читабельного представления данных в дальнейшем можно поменять их тип на `int`.
- пропуски данных для столбцов `days employed` и `total_income` в количестве 21525-19 351 = 2174 что составляет 10% от общего числа записей.
- в 47 записях фигурирует отрицательное количество детей (-1), что невозможно, у 76 клиетов в семье 20 детей (что малоправдоподобно), записи с -1 ребенком заменены на записи с 1. В дальнейшем перепроверить достоверность информации по клиентам с 20 детьми.
- общий трудовой стаж не может быть отрицательным числом. Из-за большого количества таких записей (более 70% от всей информации) можно предположить что при выгрузке данных имела места техническая ошибка. Для дальнейшего анализа информации  отрицательные значения в `days_employed` заменены на положительные.
- 101 запись содержит возраст клиента равный 0. Скорее всего, в данном случае 0 использовался для заполнения недостающих данных о возрасте. Для корректировки данных, заменили 0 в данных записях на значение медианы возраста клиентов.
- колонка `education` содержит дубликаты (написанный в разном регистре тип образования).
- выявлен клиент с загадочным полом XNA, заменен на 'unknown'.
- для обобщения целей кредита колонка `purpose` будет лемматизирована.


##  Предобработка данных

### Обработка пропусков

#### Общая информация о пропусках
Посчитаем количество пропусков для каждого столбца:

In [22]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**Вывод** Как мы видим, пропущенные значения в количественных переменных для столбцов `days_employed` и `total_income`. Для банковских данных это действительно странно, поскольку одним из необходимых документов при обращении за кредитом является справка о доходах, как и записи из трудовой (для определения стажа). Возможно пропуски обусловлены нежеданием клиента предоставлять данную инфрмацию, либо технической проблемой при выгрузке данных.

#### Обработка пропусков в `days_employed`

Поскольку в столбце представлены количественные значения, и нам очень сильно хочется их заполнить, то мы будем их заполнять медианным значением. Для начала напомним характеристики данного столбца:

In [23]:
df['days_employed'].describe()

count     19351.000000
mean      66914.728907
std      139030.880527
min          24.141633
25%         927.009265
50%        2194.220567
75%        5537.882441
max      401755.400475
Name: days_employed, dtype: float64

Мы видим, что разброс в данных весьма значителен - от 24 до 401755 дней. При этом трудовой стаж в днях для клиента проработавшего 60 лет с учетом того , что в году в среднем 250 рабочих дней = 250 * 60 = 15000 дней. Таким образом, значения в `days_employed` вряд ли могут больше 15000, возможно произошла ошибка и стаж для некоторых записей представлен не в виде дней, а в виде часов. При отсутствии возможности уточнить данную информацию заменим аномально высокие значения в `days_employed` на результат их деления на 24:

In [24]:
df['days_employed'] = df['days_employed'].apply(lambda x: x/24 if x > 15000 else x)
df['days_employed'].describe()

count    19351.000000
mean      4631.293458
std       5348.688897
min         24.141633
25%        925.259012
50%       2190.296191
75%       5520.672818
max      16739.808353
Name: days_employed, dtype: float64

Поскольку разброс значений выборки значителен мы не будем брать общую медиану. Для начала сгруппируем данные по параметру, оказывающий наибольшее влияние на общий трудовой стаж, и затем найдем медиану для каждой группы. По моему мнению, параметр, оказывающий наибольшее влияние на трудовой стаж `days_employed` - это возраст клиента`dob_years`. Для группировки данных по возрасту уже на данном этапе проведем категоризацию по возрастным группам, используя метод `cut()` или `apply()`. Напомним, что в п.1.3.3 мы уже заменили некорректные данные по возрасту (там, где возраст равен 0):

In [25]:
df['group_age'] = pd.cut(df['dob_years'], [18, 35, 55, 65, 75], labels = ['молодой', 'взрослый', 'предпенсионный', 'пенсионный'])
df.head()
# То же самое, но с созданием функции и использованием apply
# def split_group_age(age):
# ''' Делит клиетов по возрастным группам'''
#     if age <= 35:
#         return 'молодой'
#     elif 35 < age <= 55:
#         return 'взрослый'
#     elif 55 < age <= 65:
#         return 'предпенсионный'
#     else:
#         return 'пенсионный'
#df['group_age'] = df['dob_years'].apply(split_group_age)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,group_age
0,1,8437.673028,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,взрослый
1,1,4024.803754,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,взрослый
2,0,5623.422610,33.0,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,молодой
3,3,4124.747207,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,молодой
4,0,14177.753002,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,взрослый


Заполним недостающие значения в соответсвии со  значением медианы `days_employed`, найденным после группировки данных по `group_age` методом `transform` (закомментирован) или методом loc:

In [26]:
#df['days_employed'] = df['days_employed'].fillna(df.groupby('group_age')['days_employed'].transform("median"))

for age in df['group_age'].unique():
    median = df.loc[df['group_age'] == age, 'days_employed'].median()
    df.loc[(df['group_age'] == age) & df['days_employed'].isna(), 'days_employed'] = median

In [27]:
test = df.copy()

Проверим успешно ли мы заполнили пропущенные значения в столбце `days_employed`:

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null float64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
group_age           21525 non-null category
dtypes: category(1), float64(3), int64(4), object(5)
memory usage: 2.0+ MB


**Вывод** Недостающие данные в столбце `days_employed` заменены на медиану, вычисленную при группировки данных по созданному при категоризации столбцу `group_age`

#### Обработка пропусков в `total_income`

Аналогичным образом проводим заполнение недостающих значений в `total_income`  для этого:
- определяем основные характеристики `total_income`
- определяем показатель, который в большей степени влияет на `total_income` - `income_type`
- находим медиану для каждой группы данных
- применяем метод `loc` для заполнения пропусков, сгрупированных по `income_type` (либо методом `transform` (закоментирован))
- определяем количество пропущенных данных после заполнения:

In [29]:
display(df['total_income'].describe())
for inc_type in df['income_type'].unique():
    median = df.loc[df['income_type'] == inc_type, 'total_income'].median()
    df.loc[(df['total_income'].isna()) & (df['income_type'] == inc_type), 'total_income'] = median
df.info()
#df['total_income'] = df['total_income'].fillna(df.groupby('income_type')['total_income'].transform("median"))
#df.info()

count    1.935100e+04
mean     1.674223e+05
std      1.029716e+05
min      2.066726e+04
25%      1.030532e+05
50%      1.450179e+05
75%      2.034351e+05
max      2.265604e+06
Name: total_income, dtype: float64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null float64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
group_age           21525 non-null category
dtypes: category(1), float64(3), int64(4), object(5)
memory usage: 2.0+ MB


**Вывод** Недостающие данные в столбце `total_income` заменены на медиану, вычисленную при группировки данных по созданному при категоризации столбцу `income_type`

### Замена типа данных

В данном случае целесообразно заменить тип данных в двух столбцах `days_employed` и `total_income`, которые имеют вещественный тип данных  `float`, в данном примере у нас нет необходимости в точности данных до 6 цифры после запятой. Использование `astype('int')` будет отбрасывать дробную часть числа после точки. Для более корректного преобразования для начала округлим значения `float`, используя метод `round()`, а затем приведем его  к `int`

In [30]:
df['days_employed'] = df['days_employed'].round(0).astype('int')

In [31]:
df['total_income'] = df['total_income'].round(0).astype('int')

Убедимся, что замена типов данных прошла успешно:

In [32]:
df.dtypes

children               int64
days_employed          int32
dob_years            float64
education             object
education_id           int64
family_status         object
family_status_id       int64
gender                object
income_type           object
debt                   int64
total_income           int32
purpose               object
group_age           category
dtype: object

In [33]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,group_age
0,1,8438,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253876,покупка жилья,взрослый
1,1,4025,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,взрослый
2,0,5623,33.0,Среднее,1,женат / замужем,0,M,сотрудник,0,145886,покупка жилья,молодой
3,3,4125,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267629,дополнительное образование,молодой
4,0,14178,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,взрослый


**Вывод**
Мы преобразовали данные в столбцах `days_employed` и `total_income` к наиболее оптимальному для понимания и чтения виду, при этом при преобразовании дробная часть не откидывалась, а округлялась.

### Обработка дубликатов

#### Полные дубликаты

Найдем количество полных дубликатов:

In [34]:
df.duplicated().sum()

54

**Вывод** Мы не удаляем дубликаты, исходя из возможности клиента иметь несколько заявок на кредит. При отсутвии дополнительной информации (даты одобрения кредита, суммы) мы не можем быть уверенными что найденные дубликаты является в полном смысле полными дубликатами.

#### Неявные дубликаты

Как мы видели ранее, для столбца `education` характерны одни и те же значения, записанные в разном регистре:

In [35]:
df['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Переведем все значения данного столбца в нижний регистр и убедимся в правильности изменений:

In [36]:
df['education'] = df['education'].str.lower()
print(df['education'].unique())
df.sample(5)

['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,group_age
3338,1,1222,50.0,среднее,1,женат / замужем,0,M,сотрудник,0,119971,свой автомобиль,взрослый
12314,0,1680,31.0,среднее,1,в разводе,3,M,компаньон,0,84479,покупка жилой недвижимости,молодой
17200,0,499,30.0,высшее,0,женат / замужем,0,F,сотрудник,0,216101,дополнительное образование,молодой
16815,0,15746,64.0,высшее,0,вдовец / вдова,2,F,пенсионер,0,89179,сделка с автомобилем,предпенсионный
20066,2,206,26.0,среднее,1,женат / замужем,0,F,сотрудник,0,91514,строительство собственной недвижимости,молодой


**Вывод** В столбце `education` значения были приведены к общему виду (lowercase). В остальных столбцах категориальные данные записаны корректно.

### Лемматизация

Лемматизация данных должна быть проведена для столбца `purpose`. Если мы посмотрим на уникальные значения данного столбца, то увидим, что клиенты по-разному формулировали одну и ту же цель получения кредита:

In [37]:
purposes_unique = df['purpose'].unique()
purpose =  pd.DataFrame( data = purposes_unique, columns = ['purpose'] )
m = Mystem()
def find_purpose(cell_value):
    lemmas = m.lemmatize(cell_value)
    for lemma in lemmas:
        if 'жилье' in lemma or 'недвижимость' in lemma :   
            return 'недвижимость'
        elif 'образование' in lemma:
            return 'образование'
        elif 'свадьба' in lemma:
            return 'свадьба'
        elif 'автомобиль' in lemma:
            return 'автомобиль'

purpose['purpose_general'] = purpose['purpose'].apply(find_purpose)
purpose.head()

,purpose,purpose_general
0,покупка жилья,недвижимость
1,приобретение автомобиля,автомобиль
2,дополнительное образование,образование
3,сыграть свадьбу,свадьба
4,операции с жильем,недвижимость


In [38]:
df.merge(purpose, on = 'purpose').head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,group_age,purpose_general
0,1,8438,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253876,покупка жилья,взрослый,недвижимость
1,0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145886,покупка жилья,молодой,недвижимость
2,0,926,27.0,высшее,0,гражданский брак,1,M,компаньон,0,255764,покупка жилья,молодой,недвижимость
3,0,1549,48.0,среднее,1,женат / замужем,0,F,компаньон,0,157246,покупка жилья,взрослый,недвижимость
4,0,414,41.0,среднее,1,женат / замужем,0,M,госслужащий,0,118552,покупка жилья,взрослый,недвижимость


Из всех различных целей мы можем выделить кредит на **жилье** или **недвижимость**, кредит на проведение **свадьбы**, кредит на покупку **автомобиля**, кредит на **образование**.  Таким образом, мы нашли леммы, по которым будем лемматизировать значения `purpose`. Поскольку при использовании `Mystem` леммы имеют словарное отображение, то можно указывать не основу слова, а его словарное представление:

In [39]:
#df = df.merge(purpose, on = 'purpose', sort = false)     changes order of rows
#df = pd.merge_ordered(df, purpose, on='purpose')         changes order of rows
#pd.merge(df, purpose, on = 'purpose', sort = False)      changed order of rows
purpose_index = purpose.set_index('purpose')
purpose_index
df = df.join(purpose_index, on = 'purpose', rsuffix = 'purpose_general')
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,group_age,purpose_general
0,1,8438,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253876,покупка жилья,взрослый,недвижимость
1,1,4025,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,взрослый,автомобиль
2,0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145886,покупка жилья,молодой,недвижимость
3,3,4125,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267629,дополнительное образование,молодой,образование
4,0,14178,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,взрослый,свадьба


Посмотрим на количество значений в столбце `purpose_general`

In [40]:
df['purpose_general'].value_counts()

недвижимость    10840
автомобиль       4315
образование      4022
свадьба          2348
Name: purpose_general, dtype: int64

Таким образом, мы уверены что для каждой записи в таблице есть значение в столбце `purpose_general`

**Вывод** Мы лемматизировали данные в столбце `purpose`, выбрав в качестве лемм 4 основные категории кредитов: жилье(недвижимость), свадьба, образование, автомобиль и дополнили исходную таблицу столбцом `purpose_general`, позволяющим увидеть общую цель кредита.


### Категоризация данных

#### Категоризация данных по возрастным группам

Ввиду необходимости категоризация данных по возрасту клиентов была проведена ранее с созданием столбца `group_age`

In [41]:
df['group_age'].value_counts()

взрослый          10542
молодой            6594
предпенсионный     3684
пенсионный          705
Name: group_age, dtype: int64

#### Категоризация данных по уровню дохода

Для ответа на один из вопросов пункта 3 проведем категоризация данных по `total_income`, для начала определим диапазон значений `total_income`, затем напишим функцию и применим метод `apply` (`cut`, конечно быстрее, но уж ладно):

In [42]:
df['total_income'].describe()

count    2.152500e+04
mean     1.652253e+05
std      9.804367e+04
min      2.066700e+04
25%      1.077980e+05
50%      1.425940e+05
75%      1.955500e+05
max      2.265604e+06
Name: total_income, dtype: float64

In [43]:
def split_group_income(income):
    """делит клиетов на группы по уровню дохода"""
    if income <= 30000:
        return 'poverty'
    elif 30000 < income <= 55000:
        return 'lower middle class'
    elif 55000 < income <= 100000:
        return 'middle class'
    elif 100000 < income <= 300000:
        return 'higher middle class'
    elif 300000 < income <= 1000000:
        return 'rich'
    return 'kill the thief!'

df['group_income'] = df['total_income'].apply(split_group_income)


Проверим значения в новом столбце:

In [44]:
df['group_income'].value_counts()

higher middle class    15579
middle class            3883
rich                    1458
lower middle class       558
kill the thief!           25
poverty                   22
Name: group_income, dtype: int64

**Вывод** Была произведена категоризация данных по `total_income` с получением нового столбца `group_income`

#### Классификация по типу

Выведим первые пять записей исходной таблицы:

In [45]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,group_age,purpose_general,group_income
0,1,8438,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253876,покупка жилья,взрослый,недвижимость,higher middle class
1,1,4025,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,взрослый,автомобиль,higher middle class
2,0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145886,покупка жилья,молодой,недвижимость,higher middle class
3,3,4125,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267629,дополнительное образование,молодой,образование,higher middle class
4,0,14178,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,взрослый,свадьба,higher middle class


Мы видим, что столбцы `family_status_id` и `education_id` можно вынести в отдельные "словари", поскольку при их наличии в основной таблице усложняется визуальная работа с таблицей, увеличивается размер файла и время обработки данных. Создадим таблицу с основной информацией и два "словаря", в "словарях" удалим все дубликаты и переустановим индекс:

In [46]:
main_inf = df[['children','days_employed', 'dob_years', 'education_id', 'family_status_id', 'gender', 'income_type', 'debt', 'total_income', 'purpose', 'group_age', 'purpose_general']]
main_inf.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,group_age,purpose_general
0,1,8438,42.0,0,0,F,сотрудник,0,253876,покупка жилья,взрослый,недвижимость
1,1,4025,36.0,1,0,F,сотрудник,0,112080,приобретение автомобиля,взрослый,автомобиль
2,0,5623,33.0,1,0,M,сотрудник,0,145886,покупка жилья,молодой,недвижимость
3,3,4125,32.0,1,0,M,сотрудник,0,267629,дополнительное образование,молодой,образование
4,0,14178,53.0,1,1,F,пенсионер,0,158616,сыграть свадьбу,взрослый,свадьба


In [47]:
family_status_dict = df[['family_status', 'family_status_id']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
family_status_dict

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


In [48]:
education_dict = df[['education', 'education_id']]
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
family_status_dict

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


**Вывод** Мы создали два "словаря" для `education` и `family_status` - `education_dict` и `family_status_dict` соответственно, таким образом "разгрузив" основную таблицу `main_inf`, а также категоризировали данные по возрастным группам

## Ответьте на вопросы

Для ответов на вопросы мы будем использовать методы `merge` для объединения искомых данных и `pivot_table` с агрегированными функциями `count` и `sum`, покольку значения в столбце `debt` представлены единицами и нулями. Путем деления столбца `sum` (количество задолженностей) на столбец `count` (общее число клиентов) мы найдем процент задолженности

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [49]:
without_kids = df[df['children'] == 0][['debt']].agg(['sum', 'count'])
with_kids = df[df['children'] != 0][['debt']].agg(['sum', 'count'])
df_children = pd.merge(without_kids, with_kids, left_index = True, right_index = True, suffixes=('_without_kids', '_withkids'))
df_children.loc['количество задолжников, %'] = df_children.iloc[0] / df_children.iloc[1]*100
display(df_children)
#ну или с категоризацией:
def have_kids(kid):
    if kid == 0:
        return 'without kids'
    return 'with kids'
df['kids'] = df['children'].apply(have_kids)
df_pivot_kids = df.pivot_table(index = 'kids', values = 'debt', aggfunc = ['sum', 'count'])
df_pivot_kids['количество задолжников, %'] = df_pivot_kids.iloc[:, 0]/df_pivot_kids.iloc[:,1]*100
df_pivot_kids.sort_values(by = 'количество задолжников, %', ascending = False)

,debt_without_kids,debt_withkids
sum,1063.000000,678.000000
count,14149.000000,7376.000000
"количество задолжников, %",7.512898,9.191974


,sum,count,"количество задолжников, %"
,debt,debt,
kids,,,
with kids,678,7376,9.191974
without kids,1063,14149,7.512898


**Вывод**  В целом, клиенты, неимеющие детей, возвращают кредиты в срок чаще, чем клиенты, имеющие детей. При отсутствии детей у клиента меньше непредвиденных расходов и он более свободен в распределении своих финансовых поступлений с учетом необходимости оплаты кредита в срок.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [50]:
df_pivot_family = df.pivot_table(index = 'family_status', values = 'debt', aggfunc = ['sum', 'count'])
df_pivot_family['количество задолжников, %'] = df_pivot_family.iloc[:, 0]/df_pivot_family.iloc[:,1]*100
df_pivot_family.sort_values(by = 'количество задолжников, %', ascending = False)

,sum,count,"количество задолжников, %"
,debt,debt,
family_status,,,
Не женат / не замужем,274,2813,9.740491
гражданский брак,388,4177,9.288963
женат / замужем,931,12380,7.520194
в разводе,85,1195,7.112971
вдовец / вдова,63,960,6.562500


**Вывод** Чаще задолжниками становятся неженатые, незамужние люди, а также люди, имеющие гражданский брак. Наиболее высок уровень возврата кредитов у вдовцов/вдов.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [51]:
df_pivot_income = df.pivot_table(index = 'group_income', values = 'debt', aggfunc = ['sum', 'count'])
df_pivot_income['количество задолжников, %'] = df_pivot_income.iloc[:, 0]/df_pivot_income.iloc[:,1]*100
df_pivot_income.sort_values(by = 'количество задолжников, %', ascending = False)

,sum,count,"количество задолжников, %"
,debt,debt,
group_income,,,
poverty,2,22,9.090909
higher middle class,1281,15579,8.222607
middle class,319,3883,8.215297
kill the thief!,2,25,8.000000
rich,104,1458,7.133059
lower middle class,33,558,5.913978


**Вывод**  Неплательщики кредитов  - бедные клиеты с зарплатой до 30000 руб, в то время как люди с доходом от 30000 до 55000 руб. чаще остальных категорий выплачивают кредит в срок. Я удивляюсь нашим олигархам, которые с доходом более 1 млн руб вообще имеют задолженность.

- Как разные цели кредита влияют на его возврат в срок?

In [52]:
df_pivot_purpose = df.pivot_table(index = 'purpose_general', values = 'debt', aggfunc = ['sum', 'count'])
df_pivot_purpose['количество задолжников, %'] = df_pivot_purpose.iloc[:, 0]/df_pivot_purpose.iloc[:,1]*100
df_pivot_purpose.sort_values(by = 'количество задолжников, %', ascending = False)

,sum,count,"количество задолжников, %"
,debt,debt,
purpose_general,,,
автомобиль,403,4315,9.339513
образование,370,4022,9.199403
свадьба,186,2348,7.921635
недвижимость,782,10840,7.214022


**Вывод** Люди, берущие кредит на недвежимость, более ответственные кредитоплательщики, в то время как клиенты покупающие автомобиль менее часто отдают кредит в срок.

## Общий вывод

1. Корректность данных:
- в 47 записях фигурирует отрицательное количество детей (-1), что невозможно, у 76 клиетов в семье 20 детей (что малоправдоподобно), записи с -1 ребенком заменены на записи с 1. В дальнейшем перепроверить достоверность информации по клиентам с 20 детьми.
- общий трудовой стаж не может быть отрицательным числом. Из-за большого количества таких записей (более 70% от всей информации) можно предположить что при выгрузке данных имела место техническая ошибка. Для дальнейшего анализа информации заменили отрицательные значения в days_employed на положительные
- 101 запись содержит возраст клиента равный 0. Скорее всего, в данном случае 0 использовался для заполнения недостающих данных о возрасте. Для корректировки данных, заменили 0 в данных записях на значение медианы возраста клиентов
- колонка education содержит дубликаты (написанный в разном регистре тип образования), осуществлена корректировка с переводом значений в нижний регистр 
- выявлен клиент с загадочным полом XNA, заменен на 'unknown'
- для обобщения целей кредита колонка purpose лемматизирована по леммам 'жилье', 'недвижимость', 'автомоблиль', 'свадьба', 'образование'.
- недостающие данные в столбцах `days_employed` и `total_income` заменены на медиану, вычисленную при группировки данных по параметру, оказывающему наибольшее влияние на вышеуказанные значения
- мы преобразовали данные в столбцах `days_employed` и `total_income` к наиболее оптимальному для понимания и чтения виду (`int`), при этом при преобразовании дробная часть не откидывалась, а округлялась.
- мы не удаляем дубликаты, исходя из возможности клиента иметь несколько заявок на кредит, при отсутствии дополнительной информации (даты одобрения кредита, суммы) мы не можем быть уверенными, что найденные дубликаты является в полном смысле полными дубликатами.
- произведена категоризация данных по `total_income` с получением нового столбца в таблице `group_income`
- мы создали два "словаря" для `education` и `family_status` - `education_dict` и `family_status_dict` соответственно, таким образом "разгрузив" основную таблицу `main_inf`, а также категоризировали данные по возрастным группам
2. Ответы на вопросы
- в целом клиенты, неимеющие детей возвращают кредиты в срок чаще, чем клиенты, имеющие детей, за исключением клиентов с 5 детьми, которые всегда возвращают кредиты в срок (хотя в виду очень небольшого числа таких клиентов - всего 9, данная тенденция не является абсолютно доказанной). Клиенты с тремя детьми возвращают кредиты в срок чаще, чем клиенты с одним или двумя детьми. Злостные неплательщики - клиента с 20 детьми (ха-ха-ха, им бы руки успеть помыть, не то, что в банк добежать кредит вернуть), опять же возникает вопрос о корректности выгруженных в df данных.
- чаще задолжниками становятся неженатые, незамужние люди, а также люди, имеющие гражданский брак. Наиболее высок уровень возврата кредитов для вдовцов/вдов.
- неплательщики кредитов - бедные клиенты с зарплатой до 30000 руб, в то время как люди с доходом от 30000 до 55000 руб. чаще остальных категорий выплачивают кредит в срок. Я удивлена нашими олигархами, которые с доходом более 1 млн руб вообще имеют задолженность.
- люди, берущие кредит на недвежимость, более ответственные кредитоплательщики, в то время как клиенты, покупающие автомобиль, реже отдают кредит в срок.
- если Вы хотите взять кредит на автомобиль, Вы single, но с 20 детьми, при этом Ваша зарплата либо менее 30000 руб, либо более 1 млн.руб - одумайтесь, скорее всего Вы его не вернете